# oneM2M - Introduction to Basic Resources and Interactions
This notebook shows the basic interactions with a CSE using REST calls. Examples include:

- Retrieve &lt;CSEBase> resource
- Create an &lt;AE> resource
- Create a &lt;Container> resource
- Create one or more &lt;ContentInstance> resources
- Retrieve the latest &lt;ContentInstance> resource
- Update the &lt;Container> resource
- Retrieve the &lt;Container> resource
- Delete the &lt;AE> resource


## Configurations
This section defines configurations for the REST calls. Please adapt them to your environment accordingly.

In [ ]:
url        = 'http://localhost:8080/in-name'     # The URL of the CSE we want to connect to
originator = 'admin:admin'                       # Originator ID to access the CSE

## Intitialization
The section does import necessary modules and defines a helper function used in this notebook.

In [ ]:
import requests, json

# Tidy-print a response header.
def printResponse(r):
    print(str(r.status_code) + ' (' + r.reason + ')')
    print('========== Headers ==========')
    for h in r.headers:
        print(h + ':' + r.headers[h])
    if r.text:
        print('\n========== Body ==========')
        print(r.text)
    

## Retrieve the &lt;CSEBase> Resource
This first example retrieves the &lt;CSEBase> resource from the CSE. This resource represents the root of the CSE itself. all other resources are located beneath this root resource.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,                # Set the originator
    'X-M2M-RI'     : '1',                       # Unique request identifier
    'Accept'       : 'application/json',        # Response shall be JSON
    'Content-Type' : 'application/json;ty=2'    # Content is JSON, and represents an <AE> resource
    
}
response = requests.get(url, headers=headers)   # GET request
printResponse(response)

The response contains a lot of informations about the CSE itself:

- **rn** : A unique ResourceName. If left out in creation requests, the CSE assigns a unique resource name on its own.
- **ty** : Resource type.
- **ri** : Resource identifier. This uniquely identifies a resource in a CSE.
- **ct, lt** : Creation and last motification time and dates of a resource.
- **acpi** : A list of &lt;AccessControlPolicy> resources, which specify access control and rights to a resource.
- **cst** : The type of CSE (IN, MN etc)
- **csi** : The CSE Identifier.
- **srt** : A list of Resource Types supported by this CSE.
- **poa** : The Point of Access of the CSE.

## Create an &lt;AE> Resource
This example creates a new &lt;AE> resource in the CSE. &lt;AE>'s represent applications or services.

Creating this (or other) resource is done using a POST request and with providing a couple of mandatory attributes:

- **rn** : see above
- **api** : Application Identifier. An Application Identifier uniquely identifies an M2M Application in a given context.
- **rr** : Request Reachability. This attribute indicates whether a resource can receive requests.

Note, that the request target is the &lt;CSEBase> resource. All create requests target a parent resource.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '2',                      # Unique request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=2'   # Content is JSON, and represents an <AE> resource
}

data = '''
{
  "m2m:ae": {
    "rn": "myAE",
    "api": "AE",
    "rr": true
  }
}
'''

response = requests.post(url, headers=headers, data=data)    # CREATE request
printResponse(response)

The response introduces some new attributes:

- **pi** : This is the identifier of the parent resource.
- **et** : Expiration time/date after which the CSE will delete the resource.
- **aei** : An Application Entity Identifier uniquely identifies an AE globally.


## Create a &lt;Container> Resource
In this section we add a &lt;Container> resource to the &lt;AE> resource. A &lt;Container> represents a data point that can hold a configurable number of dsta instances. A &lt;Container> may also hold a sub-containers.

If created with no further attributes, the CSE will assign defaults to the &lt;Container> resource.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '3',                      # Unique request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=3'   # Content is JSON, and represents an <Container> resource
}

data = '''
{
  "m2m:cnt": {
    "rn":"myContainer"
  }
}
'''

response = requests.post(url + '/myAE', headers=headers, data=data)    # CREATE request
printResponse(response)

The following attributes are used with the &lt;Container> resource:

- **st** : The State Tag is incremented every time the resource is modified.
- **mni** : Maximum number of direct data instances in the &lt;Container> resource.
- **mbs** : Maximum size in bytes of data.
- **mia** : Maximum age of a direct data instances in the &lt;Container> resource.
- **cni** : Current number of direct data instances in the &lt;Container> resource.
- **cbs** : Current size in bytes of data.
- **ol** : Resource identifier of a virtual resource that points to the oldest data instance of the &lt;Container> resource.
- **la** : Resource identifier of a virtual resource that points to the latest data instance of the &lt;Container> resource.

## Add a &lt;ContentInstance> to the &lt;Container>

Now, we add an actual value to the *myContainer* &lt;Container>. 

These attributes are part of the request:

- **cnf** : This specifies the content format. It specifies the media type as well as an encoding type.
- **con** : This is the actual content (ie. the value) that will be stored in the &lt;Container resource.

&lt;ContentInstance>'s can only be added and read, but not updated or deleted.

**Note**: You can execute the following code as often as you like in order to create more &lt;ContentInstance> resources.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,               # Set the originator
    'X-M2M-RI'     : '4',                      # Unique request identifier
    'Accept'       : 'application/json',       # Response shall be JSON
    'Content-Type' : 'application/json;ty=4'   # Content is JSON, and represents an <ContentInstance> resource
}

data = '''
{
  "m2m:cin": {
    "cnf": "text/plain:0",
    "con": "Hello, World!"
  }
}
'''

response = requests.post(url + '/myAE/myContainer', headers=headers, data=data)    # CREATE request
printResponse(response)

- **cs** : This attribute contains the size of the content of the **con** attribute.

## Retrieve the latest &lt;ContentInstance> resource
This request will retrieve the latest data instance from the &lt;Container>.

## Update the &lt;Container Resource


## Check the &lt;Container> resource
Retrieve the &lt;Container> resource to see

## Delete the &lt;AE> Resource and all its Sub-Resources
This section finally deletes the &lt;AE> resource and all its sub-resources from the CSE.

In [ ]:
headers = {
    'X-M2M-Origin' : originator,                  # Set the originator
    'Accept'       : 'application/json',          # Response shall be JSON
    'Content-Type' : 'application/json;ty=2'      # Content is JSON, and represents an <AE> resource
}

response = requests.delete(url + '/myAE', headers=headers)   # DELETE request
printResponse(response)